# Use Case 3: Associating Clinical Variables with Acetylation

For this use case we will show how to analyze the acetylation data with a clinical attribute. We will use the clinical attribute "Histologic_type", but you can apply the processes shown here to many other clinical attributes. Our goal is to identify which acetylation sites differ significantly in frequency between non-tumor, serous and endometrial cells.

# Step 1: Import Packages and Load Data

We will start by importing the data analysis tools we need, importing the cptac package, and loading the Endometrial dataset.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import seaborn as sns
import math
import cptac
en = cptac.Endometrial()

Welcome to the cptac data service package. To view available datasets,
enter cptac.list_data(). To access a specific data set, load the
dataset and assign it to a variable using 'cptac.NameOfDataset()',
e.g. 'en = cptac.Endometrial()'

******
Version: 0.4.3
******
You have loaded the cptac endometrial dataset. To view available
dataframes, call the dataset's list_data() method. To view available
functions for accessing and manipulating the dataframes, call its
list_api() method.
endometrial data version: 2.1

Loading acetylproteomics data...
Loading clinical data...
Loading CNA data...
Loading definitions data...
Loading miRNA data...
Loading phosphoproteomics_gene data...
Loading phosphoproteomics_site data...
Loading proteomics data...
Loading somatic data...
Loading somatic_binary data...
Loading transcriptomics_circular data...
Loading transcriptomics_linear data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation 

# Step 2: Choose Clinical Attribute and Merge Dataframes

If you want to see all the different variables available in the clinical data, you can call <code>en.get_clinical().columns</code>. For this use case, we will use the 'Histologic_type' clinical attribute in order to find differences in acetylation sites between "endometrioid" and "serous" cancer cells. 

In [2]:
#Set desired attribute to variable 'clinical_attribute'
clinical_attribute = "Histologic_type"

Here we will merge our desired clinical attribute with our acetylation dataframe using the <code>en.compare_omics</code> method.

In [3]:
#Merge attribute with acetylation dataframe
clinical_and_acetylation = en.append_metadata_to_omics(metadata_df_name="clinical", omics_df_name="acetylproteomics", metadata_cols=clinical_attribute)
clinical_and_acetylation.head()

Histologic_type  A2M-K1168_acetylproteomics  \
Sample_ID                                               
S001         Endometrioid                         NaN   
S002         Endometrioid                         NaN   
S003         Endometrioid                         NaN   
S005         Endometrioid                         NaN   
S006               Serous                         NaN   

           A2M-K1176_acetylproteomics  A2M-K135_acetylproteomics  \
Sample_ID                                                          
S001                            1.080                        NaN   
S002                            0.477                        NaN   
S003                              NaN                        NaN   
S005                           -0.608                        NaN   
S006                            1.630                        NaN   

           A2M-K145_acetylproteomics  A2M-K516_acetylproteomics  \
Sample_ID                                                         
S001                             NaN                        NaN   
S002                             NaN                        NaN   
S003                             NaN                        NaN   
S005                             NaN                     -0.919   
S006                             2.4                        NaN   

           A2M-K664_acetylproteomics  A2M-K682_acetylproteomics  \
Sample_ID                                                         
S001                             NaN                        NaN   
S002                             NaN                        NaN   
S003                             NaN                        NaN   
S005                             NaN                        NaN   
S006                             NaN                       1.26   

           AACS-K391_acetylproteomics  AAGAB-K290_acetylproteomics  ...  \
Sample_ID                                                           ...   
S001                              NaN                        0.461  ...   
S002                              NaN                        1.770  ...   
S003                              NaN                       -0.815  ...   
S005                              NaN                          NaN  ...   
S006                              NaN                        0.205  ...   

           ZSCAN31-K215_acetylproteomics  ZSCAN32-K659_acetylproteomics  \
Sample_ID                                                                 
S001                                 NaN                            NaN   
S002                              -0.104                       -0.80300   
S003                                 NaN                            NaN   
S005                              -0.457                       -0.00175   
S006                                 NaN                        0.41100   

           ZW10-K634_acetylproteomics  ZYX-K24_acetylproteomics  \
Sample_ID                                                         
S001                              NaN                       NaN   
S002                              NaN                    -0.988   
S003                              NaN                    -0.459   
S005                            -0.33                       NaN   
S006                              NaN                       NaN   

           ZYX-K25_acetylproteomics  ZYX-K265_acetylproteomics  \
Sample_ID                                                        
S001                            NaN                        NaN   
S002                         -0.343                     -0.307   
S003                         -1.170                        NaN   
S005                         -0.537                        NaN   
S006                         -0.358                        NaN   

           ZYX-K272_acetylproteomics  ZYX-K279_acetylproteomics  \
Sample_ID                                                         
S001                             NaN                        NaN   
S002   

# Step 3: Make Different Dataframes and Establish Threshold

In [4]:
#Show possible variations of Histologic_type
clinical_and_acetylation[clinical_attribute].unique()

array(['Endometrioid', 'Serous', nan], dtype=object)

In this step, we will make two different dataframes for "Endometrioid" and "Serous" cancer types, as well as fill the NaN columns with "Non-Tumor."

In [5]:
#Make dataframes with only endometrioid and only serous data in order to compare 
endom = clinical_and_acetylation.loc[clinical_and_acetylation[clinical_attribute] == "Endometrioid"]
serous = clinical_and_acetylation.loc[clinical_and_acetylation[clinical_attribute] == "Serous"]
#Here is where we set the NaN values to "Non_Tumor"
clinical_and_acetylation[[clinical_attribute]] = clinical_and_acetylation[[clinical_attribute]].fillna(value="Non_Tumor")

Now that we have our different dataframes, we want to make sure that the amount of data we are using for each site is significant. Since there are fewer patients with "serous" tumors than with "endometrioid," we will check to make sure that we have at least five values for each acetylation site that we are comparing that have a measurement of intensity for serous patients.

In [6]:
num_sites_sufficient_data = 0
for num in range(1,len(serous.columns)):
    site = serous.columns[num]
    oneSite = serous[site]
    num_datapoints = oneSite.count() #Here we count the number of datapoints that we have at each site
    if num_datapoints >= 5:
        num_sites_sufficient_data += 1
print("Number of sites with sufficient data: ", num_sites_sufficient_data)
threshold = .05 / num_sites_sufficient_data
print("Multiple hypothesis adjusted threshold: ", threshold)

Number of sites with sufficient data:  3936
Multiple hypothesis adjusted threshold:  1.2703252032520325e-05


# Step 4: Compare Endometrioid and Serous Values

Now that we have a calculated threshold value, we will loop through the data and compare endometrioid versus serous data for each acetylation site. If we find a site that is significantly different, we will add it to a list.

In [7]:
significantTests = []
significantSites = []
sigSiteCount = 0
p_value_index = 1
np.warnings.filterwarnings('ignore')
for num in range(1,len(clinical_and_acetylation.columns)):
    site = clinical_and_acetylation.columns[num]
    ttestRes = scipy.stats.ttest_ind(endom[site], serous[site])#This returns a tuple, with the t-statistic as 
    #the first value and the p-value as the second(index 1). We are interested in the p-value.
    if (ttestRes[p_value_index] <= threshold): #Check if there is a significant enough difference between data points
        sigSiteCount += 1
        significantTests.append(ttestRes)
        significantSites.append(site)
print("Number of sites with statistically significant difference between endometrioid and serous values: " + str(sigSiteCount))

Number of sites with statistically significant difference between endometrioid and serous values: 8


In [8]:
print(significantSites)

['FOXA2-K280_acetylproteomics', 'FUS-K332_acetylproteomics', 'JADE3-K32_acetylproteomics', 'MCRS1-K136_acetylproteomics', 'MEAF6-K69_acetylproteomics', 'NCL-K398_acetylproteomics', 'SUPT16H-K674_acetylproteomics', 'TBL1XR1-K102_acetylproteomics']


# Step 5: Graph Results

Now that we have eight acetylation sites that differ significantly between endometrioid and serous intensities, we will graph a couple of them using a boxplot and a stripplot in order to visually see the difference, as well as compare with normal cells.

In [9]:
graphingSite = 'FOXA2-K280_acetylproteomics'
print(scipy.stats.ttest_ind(endom[graphingSite], serous[graphingSite]))
sns.boxplot(x=clinical_attribute, y=graphingSite, data = clinical_and_acetylation, showfliers = False, order = ["Non_Tumor", "Endometrioid", "Serous"])
sns.stripplot(x=clinical_attribute, y=graphingSite, data = clinical_and_acetylation, color = '.3', order = ["Non_Tumor", "Endometrioid", "Serous"])
#color = '.3' makes the dots black

Ttest_indResult(statistic=5.991702127182772, pvalue=3.89570439448459e-08)


In [10]:
graphingSite = 'TBL1XR1-K102_acetylproteomics'
print(scipy.stats.ttest_ind(endom[graphingSite], serous[graphingSite]))
sns.boxplot(x=clinical_attribute, y=graphingSite, data = clinical_and_acetylation, showfliers = False, order = ["Non_Tumor", "Endometrioid", "Serous"])
sns.stripplot(x=clinical_attribute, y=graphingSite, data = clinical_and_acetylation, color = '.3', order = ["Non_Tumor", "Endometrioid", "Serous"])

Ttest_indResult(statistic=-6.058405376720225, pvalue=2.8961545616585778e-08)
